In [8]:
from influxdb_client import InfluxDBClient, Point, WriteOptions

### MQTT

Use this notebook to send ad hoc requests to the Edge-device/Backend

In [22]:
import paho.mqtt.client as mqtt

mqtt_username = "mosquitto"
mqtt_password = "mosquitto"
mqtt_host = "18.170.70.157"
mqtt_port = 1883

topic = "test"

# The callback for when the client receives a CONNACK response from the server.
def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+str(rc))
    client.subscribe(topic)

# The callback for when a PUBLISH message is received from the server.
def on_message(client, userdata, msg):
    print(str(msg.payload.decode('utf-8')))

def on_publish(client, userdata, mid):
    print("mid: "+str(mid))

mqtt_client = mqtt.Client()
mqtt_client.on_connect = on_connect
mqtt_client.on_message = on_message
mqtt_client.on_publish = on_publish

mqtt_client.username_pw_set(username=mqtt_username, password=mqtt_password)
mqtt_client.connect(mqtt_host, mqtt_port, 60)

0

In [10]:
mqtt_client.publish("sensor/123/987/temp", 15.20232, qos=0)


mid: 1


In [24]:
mqtt_client.publish("external/change_protocol", 1, qos=0)


mid: 2
